In [1]:
import os
import logging
from dotenv import load_dotenv
import google.generativeai as genai
from abc import ABC, abstractmethod

load_dotenv()

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

/Users/mahmoud/agent-with-graphiti/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LLMProvider(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        pass

class GeminiProvider(LLMProvider):
    def __init__(self, model_name: str = "gemini-2.5-flash"):
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        self.model = genai.GenerativeModel(model_name)

    def generate(self, prompt: str) -> str:
        logger.debug(f"Generating response for prompt length: {len(prompt)}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                logger.debug(f"Gemini API call attempt {attempt + 1}")
                response = self.model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0,
                        max_output_tokens=500,
                        top_p=0.8,
                        top_k=40
                    )
                )

                if hasattr(response, 'text') and response.text:
                    logger.debug(f"Gemini response received: {response.text[:200]}...")
                    return response.text

                logger.warning(f"Empty response from Gemini (attempt {attempt + 1})")

            except Exception as e:
                logger.error(f"Gemini API error (attempt {attempt + 1}): {e}")
                if attempt == max_retries - 1:
                    fallback_response = '{"thought": "Technical difficulties", "answer": "I\'m experiencing technical issues. Please try again."}'
                    logger.debug(f"Returning fallback response: {fallback_response}")
                    return fallback_response

                import time
                time.sleep(1)

        fallback_response = '{"thought": "Max retries exceeded", "answer": "Connection issues. Please try again later."}'
        logger.debug(f"Max retries exceeded, returning: {fallback_response}")
        return fallback_response


In [3]:
def create_test_prompt(query: str, history: str = "") -> str:
    return f"""You are a Customer Service agent for customer service.

Query: {query}
History: {history}

Available tools: SEARCH_PRODUCTS, GET_PRODUCT_DETAILS, CHECK_AVAILABILITY, PLACE_ORDER, GET_ORDER_HISTORY, GET_USER_PREFERENCES, UPDATE_USER_PREFERENCES, RESPOND_TO_CUSTOMER

Respond in JSON format:
{{"thought": "reasoning", "action": {{"name": "TOOL_NAME", "input": "input"}}}}
OR
{{"thought": "reasoning", "answer": "final answer"}}
"""

llm_provider = GeminiProvider()

prompt1 = create_test_prompt("I want to search for iPhone")
print(f"PROMPT:\n{prompt1}")
print("\n" + "-" * 30)

response1 = llm_provider.generate(prompt1)
print(f"JSON RESPONSE:\n{response1}")

2025-07-10 17:40:09,816 - DEBUG - Generating response for prompt length: 420
2025-07-10 17:40:09,817 - DEBUG - Gemini API call attempt 1


PROMPT:
You are a ReAct agent for customer service.

Query: I want to search for iPhone
History: 

Available tools: SEARCH_PRODUCTS, GET_PRODUCT_DETAILS, CHECK_AVAILABILITY, PLACE_ORDER, GET_ORDER_HISTORY, GET_USER_PREFERENCES, UPDATE_USER_PREFERENCES, RESPOND_TO_CUSTOMER

Respond in JSON format:
{"thought": "reasoning", "action": {"name": "TOOL_NAME", "input": "input"}}
OR
{"thought": "reasoning", "answer": "final answer"}


------------------------------


2025-07-10 17:40:11,535 - DEBUG - Gemini response received: ```json
{"thought": "The user wants to search for a product, specifically 'iPhone'. The `SEARCH_PRODUCTS` tool is appropriate for this action.", "action": {"name": "SEARCH_PRODUCTS", "input": "iPhone"...


JSON RESPONSE:
```json
{"thought": "The user wants to search for a product, specifically 'iPhone'. The `SEARCH_PRODUCTS` tool is appropriate for this action.", "action": {"name": "SEARCH_PRODUCTS", "input": "iPhone"}}
```
